# Summary

----

In [104]:
mysqld._mysqld_process

# Imports

In [13]:
%run imports.ipynb

INFO - root - Initialized logger


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-06-10 10:08:22.664922

ERROR - root - Initialized logger


In [105]:
%run mysqld.ipynb

INFO - root - Initialized logger
ERROR - root - Initialized logger
DEBUG - datapkg.daemons - 'basedir': /home/kimlab1/strokach/anaconda
INFO - root - MySQL database already running...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-06-10 20:00:18.889928


In [3]:
NOTEBOOK_NAME = 'load_data'
NOTEBOOK_NAME

'load_data'

In [4]:
CONNECTION_STRING = mysqld.get_connection_string(SCHEMA_NAME)
CONNECTION_STRING

'mysql://root:@localhost:8301/dbnsfp?unix_socket=/tmp/strokach/dbnsfp/mysql.sock'

In [5]:
db = datapkg.FileToMySQL(CONNECTION_STRING, NOTEBOOK_NAME, None, echo=False, db_engine='MyISAM')

# Extract

In [6]:
!ls ../downloads

dbNSFPv3.2a.zip


In [ ]:
!7z x -y -bd -bb0 '../downloads/dbNSFPv3.2a.zip' -o'$NOTEBOOK_NAME'

In [7]:
!ls -lh $NOTEBOOK_NAME

total 96G
-rw------- 1 strokach kimlab  78K Mar 31 00:50 dbNSFP3.2a.readme.txt
-rw------- 1 strokach kimlab 9.6G Jun  9 03:26 dbNSFP3.2a_variant.chr1
-rw------- 1 strokach kimlab 3.8G Jun  9 11:29 dbNSFP3.2a_variant.chr10
-rw------- 1 strokach kimlab 5.7G Jun  9 12:10 dbNSFP3.2a_variant.chr11
-rw------- 1 strokach kimlab 5.2G Jun  9 13:07 dbNSFP3.2a_variant.chr12
-rw------- 1 strokach kimlab 1.7G Jun  9 13:55 dbNSFP3.2a_variant.chr13
-rw------- 1 strokach kimlab 3.1G Jun  9 14:16 dbNSFP3.2a_variant.chr14
-rw------- 1 strokach kimlab 3.3G Jun  9 17:25 dbNSFP3.2a_variant.chr15
-rw------- 1 strokach kimlab 4.1G Jun  9 23:20 dbNSFP3.2a_variant.chr16
-rw------- 1 strokach kimlab 5.5G Jun 10 02:07 dbNSFP3.2a_variant.chr17
-rw------- 1 strokach kimlab 1.5G Jun 10 03:11 dbNSFP3.2a_variant.chr18
-rw------- 1 strokach kimlab 6.1G Jun 10 03:32 dbNSFP3.2a_variant.chr19
-rw------- 1 strokach kimlab 7.2G Jun  9 05:06 dbNSFP3.2a_variant.chr2
-rw------- 1 strokach kimlab 2.3G Jun 10 04:39 dbNSFP3.2a_v

# Load to database

## gene

In [ ]:
gene = pd.read_csv(
    op.join(NOTEBOOK_NAME, 'dbNSFP3.2_gene'), sep='\t', nrows=100, na_values=['.', '\\N'])
gene.columns = datapkg.format_columns(gene.columns)
display(gene.head(2))

In [ ]:
t = db.import_table(
    filename=op.abspath(op.join(NOTEBOOK_NAME, 'dbNSFP3.2_gene')),
    tablename='gene',
    na_values=['.', '\\N'],
)

In [ ]:
db.create_indexes(
    t.name,
    index_commands=[
        [('gene_name', ), True], 
        [('ensembl_gene', ), False],
        [('uniprot_acc', ), False],
        [('uniprot_id', ), False],
        [('entrez_gene_id', ), True],
        [('mim_id', ), True],
    ],
)

## gene_complete

In [ ]:
gene_complete = pd.read_csv(
    op.join(NOTEBOOK_NAME, 'dbNSFP3.2_gene.complete'), sep='\t', nrows=100, na_values=['.', '\\N'])
gene_complete.columns = datapkg.format_columns(gene_complete.columns)
display(gene_complete.head(2))

In [ ]:
t = db.import_table(
    filename=op.abspath(op.join(NOTEBOOK_NAME, 'dbNSFP3.2_gene.complete')),
    tablename='gene_complete',
    na_values=['.', '\\N']
)

In [ ]:
db.create_indexes(
    t.name,
    index_commands=[
        [('gene_name', ), True], 
        [('ensembl_gene', ), False],
        [('uniprot_acc', ), False],
        [('uniprot_id', ), False],
        [('entrez_gene_id', ), True],
        [('mim_id', ), True],
    ],
)

In [ ]:
# Try to figure out why there are one -> many, uniprot_acc -> gene_name mappings
sql_query = """\
select uniprot_acc, group_concat(distinct gene_name) gene_names
from gene_complete
group by uniprot_acc
having count(gene_name) > 1
"""
df = pd.read_sql_query(sql_query, db.engine)

In [ ]:
display(df.head())
print(df.shape)

## variant

In [10]:
variant_chr1 = pd.read_csv(
    op.join(NOTEBOOK_NAME, 'dbNSFP3.2a_variant.chr1'), sep='\t', nrows=100, na_values=['.', '\\N'])
display(variant_chr1.head(2))

,#chr,pos(1-based),ref,alt,aaref,aaalt,rs_dbSNP146,hg19_chr,hg19_pos(1-based),hg18_chr,hg18_pos(1-based),genename,cds_strand,refcodon,codonpos,codon_degeneracy,Ancestral_allele,AltaiNeandertal,Denisova,Ensembl_geneid,Ensembl_transcriptid,Ensembl_proteinid,aapos,SIFT_score,SIFT_converted_rankscore,SIFT_pred,Uniprot_acc_Polyphen2,Uniprot_id_Polyphen2,Uniprot_aapos_Polyphen2,Polyphen2_HDIV_score,Polyphen2_HDIV_rankscore,Polyphen2_HDIV_pred,Polyphen2_HVAR_score,Polyphen2_HVAR_rankscore,Polyphen2_HVAR_pred,LRT_score,LRT_converted_rankscore,LRT_pred,LRT_Omega,MutationTaster_score,MutationTaster_converted_rankscore,MutationTaster_pred,MutationTaster_model,MutationTaster_AAE,MutationAssessor_UniprotID,MutationAssessor_variant,MutationAssessor_score,MutationAssessor_score_rankscore,MutationAssessor_pred,FATHMM_score,FATHMM_converted_rankscore,FATHMM_pred,PROVEAN_score,PROVEAN_converted_rankscore,PROVEAN_pred,Transcript_id_VEST3,Transcript_var_VEST3,VEST3_score,VEST3_rankscore,MetaSVM_score,MetaSVM_rankscore,MetaSVM_pred,MetaLR_score,MetaLR_rankscore,MetaLR_pred,Reliability_index,CADD_raw,CADD_raw_rankscore,CADD_phred,DANN_score,DANN_rankscore,fathmm-MKL_coding_score,fathmm-MKL_coding_rankscore,fathmm-MKL_coding_pred,fathmm-MKL_coding_group,Eigen-raw,Eigen-phred,Eigen-raw_rankscore,Eigen-PC-raw,Eigen-PC-raw_rankscore,GenoCanyon_score,GenoCanyon_score_rankscore,integrated_fitCons_score,integrated_fitCons_score_rankscore,integrated_confidence_value,GM12878_fitCons_score,GM12878_fitCons_score_rankscore,GM12878_confidence_value,H1-hESC_fitCons_score,H1-hESC_fitCons_score_rankscore,H1-hESC_confidence_value,HUVEC_fitCons_score,HUVEC_fitCons_score_rankscore,HUVEC_confidence_value,GERP++_NR,GERP++_RS,GERP++_RS_rankscore,phyloP100way_vertebrate,phyloP100way_vertebrate_rankscore,phyloP20way_mammalian,phyloP20way_mammalian_rankscore,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons20way_mammalian,phastCons20way_mammalian_rankscore,SiPhy_29way_pi,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,1000Gp3_AC,1000Gp3_AF,1000Gp3_AFR_AC,1000Gp3_AFR_AF,1000Gp3_EUR_AC,1000Gp3_EUR_AF,1000Gp3_AMR_AC,1000Gp3_AMR_AF,1000Gp3_EAS_AC,1000Gp3_EAS_AF,1000Gp3_SAS_AC,1000Gp3_SAS_AF,TWINSUK_AC,TWINSUK_AF,ALSPAC_AC,ALSPAC_AF,ESP6500_AA_AC,ESP6500_AA_AF,ESP6500_EA_AC,ESP6500_EA_AF,ExAC_AC,ExAC_AF,ExAC_Adj_AC,ExAC_Adj_AF,ExAC_AFR_AC,ExAC_AFR_AF,ExAC_AMR_AC,ExAC_AMR_AF,ExAC_EAS_AC,ExAC_EAS_AF,ExAC_FIN_AC,ExAC_FIN_AF,ExAC_NFE_AC,ExAC_NFE_AF,ExAC_SAS_AC,ExAC_SAS_AF,ExAC_nonTCGA_AC,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AC,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AC,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AC,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AC,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AC,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AC,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AC,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AC,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AC,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AC,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AC,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AC,ExAC_nonpsych_EAS_AF,ExAC_nonpsych_FIN_AC,ExAC_nonpsych_FIN_AF,ExAC_nonpsych_NFE_AC,ExAC_nonpsych_NFE_AF,ExAC_nonpsych_SAS_AC,ExAC_nonpsych_SAS_AF,clinvar_rs,clinvar_clnsig,clinvar_trait,clinvar_golden_stars,Interpro_domain,GTEx_V6_gene,GTEx_V6_tissue
0,1,69091,A,C,M,L,NaN,1,69091,1,58954,OR4F5,+,ATG,1,0,NaN,./.,./.,ENSG00000186092,ENST00000335137,ENSP00000334393,1,0.13,0.26702,T,Q8NH21,OR4F5_HUMAN,1.0,0.0,0.02634,B,0.0,0.01257,B,0.589091,0.05577,N,1.33974,NaN,NaN,NaN,NaN,NaN,OR4F5_HUMAN,M1L,NaN,NaN,NaN,6.76,0.00529,T,-0.38,0.13435,N,NM_001005484.1,M1L,0.120,0.13350,-0.9577,0.39629,T,0.0009,0.00318,T,8.0,-1.436194,0.01840,0.003,0.443783,0.03370,0.02063,0.06083,N,AEFI,-1.394131,0.119256,0.02697,-1.535705,0.02036,0.000022,0.0355,0.487112,0.13308,0,0.573888,0.26071,0,0.573888,0.22998,0,0.564101,0.26208,0,2.31,-4.63,0.03101,-0.055,0.11668,-1.983,0.00506,0.0,0.06329,0.0,0.01567,0.2547:0.0:0.5282:0.2171,3.5592,0.07372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [11]:
variant_chr1.columns = datapkg.format_columns(variant_chr1.columns)
display(variant_chr1.head(2))

,chr,pos_1based,ref,alt,aaref,aaalt,rs_dbsnp146,hg19_chr,hg19_pos_1based,hg18_chr,hg18_pos_1based,genename,cds_strand,refcodon,codonpos,codon_degeneracy,ancestral_allele,altai_neandertal,denisova,ensembl_geneid,ensembl_transcriptid,ensembl_proteinid,aapos,sift_score,sift_converted_rankscore,sift_pred,uniprot_acc_polyphen2,uniprot_id_polyphen2,uniprot_aapos_polyphen2,polyphen2_hdiv_score,polyphen2_hdiv_rankscore,polyphen2_hdiv_pred,polyphen2_hvar_score,polyphen2_hvar_rankscore,polyphen2_hvar_pred,lrt_score,lrt_converted_rankscore,lrt_pred,lrt_omega,mutationtaster_score,mutationtaster_converted_rankscore,mutationtaster_pred,mutationtaster_model,mutationtaster_aae,mutationassessor_uniprotid,mutationassessor_variant,mutationassessor_score,mutationassessor_score_rankscore,mutationassessor_pred,fathmm_score,fathmm_converted_rankscore,fathmm_pred,provean_score,provean_converted_rankscore,provean_pred,transcript_id_vest3,transcript_var_vest3,vest3_score,vest3_rankscore,metasvm_score,metasvm_rankscore,metasvm_pred,metalr_score,metalr_rankscore,metalr_pred,reliability_index,cadd_raw,cadd_raw_rankscore,cadd_phred,dann_score,dann_rankscore,fathmmmkl_coding_score,fathmmmkl_coding_rankscore,fathmmmkl_coding_pred,fathmmmkl_coding_group,eigenraw,eigenphred,eigenraw_rankscore,eigenpcraw,eigenpcraw_rankscore,genocanyon_score,genocanyon_score_rankscore,integrated_fitcons_score,integrated_fitcons_score_rankscore,integrated_confidence_value,gm12878_fitcons_score,gm12878_fitcons_score_rankscore,gm12878_confidence_value,h1hesc_fitcons_score,h1hesc_fitcons_score_rankscore,h1hesc_confidence_value,huvec_fitcons_score,huvec_fitcons_score_rankscore,huvec_confidence_value,gerp_nr,gerp_rs,gerp_rs_rankscore,phylop100way_vertebrate,phylop100way_vertebrate_rankscore,phylop20way_mammalian,phylop20way_mammalian_rankscore,phastcons100way_vertebrate,phastcons100way_vertebrate_rankscore,phastcons20way_mammalian,phastcons20way_mammalian_rankscore,siphy_29way_pi,siphy_29way_logodds,siphy_29way_logodds_rankscore,gp3_ac,gp3_af,gp3_afr_ac,gp3_afr_af,gp3_eur_ac,gp3_eur_af,gp3_amr_ac,gp3_amr_af,gp3_eas_ac,gp3_eas_af,gp3_sas_ac,gp3_sas_af,twinsuk_ac,twinsuk_af,alspac_ac,alspac_af,esp6500_aa_ac,esp6500_aa_af,esp6500_ea_ac,esp6500_ea_af,exac_ac,exac_af,exac_adj_ac,exac_adj_af,exac_afr_ac,exac_afr_af,exac_amr_ac,exac_amr_af,exac_eas_ac,exac_eas_af,exac_fin_ac,exac_fin_af,exac_nfe_ac,exac_nfe_af,exac_sas_ac,exac_sas_af,exac_nontcga_ac,exac_nontcga_af,exac_nontcga_adj_ac,exac_nontcga_adj_af,exac_nontcga_afr_ac,exac_nontcga_afr_af,exac_nontcga_amr_ac,exac_nontcga_amr_af,exac_nontcga_eas_ac,exac_nontcga_eas_af,exac_nontcga_fin_ac,exac_nontcga_fin_af,exac_nontcga_nfe_ac,exac_nontcga_nfe_af,exac_nontcga_sas_ac,exac_nontcga_sas_af,exac_nonpsych_ac,exac_nonpsych_af,exac_nonpsych_adj_ac,exac_nonpsych_adj_af,exac_nonpsych_afr_ac,exac_nonpsych_afr_af,exac_nonpsych_amr_ac,exac_nonpsych_amr_af,exac_nonpsych_eas_ac,exac_nonpsych_eas_af,exac_nonpsych_fin_ac,exac_nonpsych_fin_af,exac_nonpsych_nfe_ac,exac_nonpsych_nfe_af,exac_nonpsych_sas_ac,exac_nonpsych_sas_af,clinvar_rs,clinvar_clnsig,clinvar_trait,clinvar_golden_stars,interpro_domain,gtex_v6_gene,gtex_v6_tissue
0,1,69091,A,C,M,L,NaN,1,69091,1,58954,OR4F5,+,ATG,1,0,NaN,./.,./.,ENSG00000186092,ENST00000335137,ENSP00000334393,1,0.13,0.26702,T,Q8NH21,OR4F5_HUMAN,1.0,0.0,0.02634,B,0.0,0.01257,B,0.589091,0.05577,N,1.33974,NaN,NaN,NaN,NaN,NaN,OR4F5_HUMAN,M1L,NaN,NaN,NaN,6.76,0.00529,T,-0.38,0.13435,N,NM_001005484.1,M1L,0.120,0.13350,-0.9577,0.39629,T,0.0009,0.00318,T,8.0,-1.436194,0.01840,0.003,0.443783,0.03370,0.02063,0.06083,N,AEFI,-1.394131,0.119256,0.02697,-1.535705,0.02036,0.000022,0.0355,0.487112,0.13308,0,0.573888,0.26071,0,0.573888,0.22998,0,0.564101,0.26208,0,2.31,-4.63,0.03101,-0.055,0.11668,-1.983,0.00506,0.0,0.06329,0.0,0.01567,0.2547:0.0:0.5282:0.2171,3.5592,0.07372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# ...

In [86]:
%%file $NOTEBOOK_NAME/_split_variant_columns.py

from itertools import chain
try:
    from itertools import izip_longest as zip_longest
except ImportError:
    from itertools import zip_longest


ensembl_columns = [
    (21, 'ensembl_transcriptid'), 
    (22, 'ensembl_proteinid'), 
    #
    (5, '_aaref'),
    (23, 'ensembl_mutation'),
    (6, '_aaalt'),
    #
    (24, 'sift_score'), 
    (26, 'sift_pred'), 
    (50, 'fathmm_score'), 
    (52, 'fathmm_pred'), 
    (53, 'provean_score'), 
    (55, 'provean_pred'),
]


uniprot_columns = [
    (27, 'uniprot_acc'), 
    (28, 'uniprot_id'),
    # 
    (5, '_aaref'),
    (29, 'uniprot_mutation'),
    (6, '_aaalt'),
    #
    (30, 'polyphen2_hdiv_score'), 
    (32, 'polyphen2_hdiv_pred'), 
    (33, 'polyphen2_hvar_score'), 
    (35, 'polyphen2_hvar_pred'),
]



def main(file, start_idx):
    """Split rows containing comma-separated lists of ensembl ids and uniprot ids into separate files."""
    ifh = open(file, 'rt')
    #
    file_ensembl = file + '_ensembl'
    file_ensembl_fh = open(file_ensembl, 'w')
    file_ensembl_fh.write(
        'variant_idx\t' + '\t'.join(c[1] for c in ensembl_columns if not c[1].startswith('_')) + '\n')
    #
    file_uniprot = file + '_uniprot'
    file_uniprot_fh = open(file_uniprot, 'w')
    file_uniprot_fh.write(
        'variant_idx\t' + '\t'.join(c[1] for c in uniprot_columns if not c[1].startswith('_')) + '\n')
    #
    next(ifh)  # skip the header
    for i, line in enumerate(ifh):
        variant_idx = str(start_idx + i)
        row = line.strip().split('\t')
        for values in iter_row(row, ensembl_columns, variant_idx):
            file_ensembl_fh.write('\t'.join(values) + '\n')
        for values in iter_row(row, uniprot_columns, variant_idx):
            file_uniprot_fh.write('\t'.join(values) + '\n')

        
def iter_row(row, columns, idx):
    """.

    Parameters
    ----------
    row : list 
        List of column values
    columns : list of tuples
        List of (column_num, column_name) tuples.
    idx : int
        Becomes the first column.
    """
    # Have to subtract 1 because indexes start from 0
    for values in zip_longest(*[row[c[0] - 1].split(',') for c in columns], fillvalue='\\N'):
        if values[0] == '\\N':
            continue
        mutation = values[2] + values[3] + values[4]
        yield chain([idx], values[:2], [mutation], values[5:])
        


if __name__ == '__main__':
    import sys
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', '--file', type=str)
    parser.add_argument('-i', '--idx', type=int)
    args = parser.parse_args()
    sys.exit(main(args.file, args.idx))

Overwriting load_data/_split_variant_columns.py


In [12]:
def load_table_to_db(suffix, dtypes=None, auto_increment=1):
    file = op.abspath(op.join(NOTEBOOK_NAME, 'dbNSFP3.2a_variant.chr{}'.format(suffix)))
    tablename = 'variant_chr{}'.format(suffix)
    
    # Replace ';' to ',' so that FIND_IN_SET works
    datapkg.run_command("sed -i 's/;/,/g' '{}'".format(file))
    
    # Create separate `_ensembl` and `_uniprot` files.
    system_command = (
        'pypy load_data/_split_variant_columns.py -f {file} -i {auto_increment}'
        .format(file=file, auto_increment=auto_increment)
    )
    p = subprocess.run(
        shlex.split(auto_increment), 
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    try:
        p.check_returncode()
    except subprocess.CalledProcessError:
        print(p.stdout)
        print(p.stderr)
        raise 


    # === Main table ===

    # Load file to database
    t = db.import_table(
        filename=file,
        tablename=tablename,
        # na_values=['\\N'],
        na_values=['.', '\\N'],
        dtypes=dtypes,
        extra_dtypes={
            'chr': sa.VARCHAR(2), 
            'hg19_chr': sa.VARCHAR(2), 'hg18_chr': sa.VARCHAR(2),
            'hg19_pos_1based': sa.INTEGER, 'hg18_pos_1based': sa.INTEGER,
        }
    )
    
    # Add an autoincrement 'idx' column
    max_id = db.add_id_column(t.name, auto_increment=auto_increment)
    
    # Create indices
    db.create_indexes(
        t.name,
        index_commands=[
            #
            [('chr', 'pos_1based', 'ref', 'alt', 
              'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            [('hg19_chr', 'hg19_pos_1based', 'ref', 'alt', 
              'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            [('hg18_chr', 'hg18_pos_1based', 'ref', 'alt', 
              'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            #
            [('rs_dbsnp146', 'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            [('clinvar_rs', 'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            #
            [('genename', 'aaref', 'aaalt', ), False],
            [('ensembl_geneid', 'aaref', 'aaalt', ), False],
            #
            [('mutationassessor_uniprotid ', 'mutationassessor_variant', 
              'chr', 'pos_1based', 'ref', 'alt', ), False],
        ],
    )


    # === Ensembl table ===

    te = db.import_table(
        filename=file + '_ensembl',
        tablename=tablename + '_ensembl',
        na_values=['\\N'],
    )
    db.add_id_column(te.name)
    db.create_indexes(
        te.name,
        index_commands=[
            [('ensembl_transcriptid', 'ensembl_mutation', 'fathmm_pred', 'fathmm_score'), True],
            [('ensembl_proteinid', 'ensembl_mutation', 'fathmm_pred', 'fathmm_score'), True],
        ],
    )


    # === Uniprot KB table ====

    tu = db.import_table(
        filename=file + '_uniprot',
        tablename=tablename + '_uniprot',
        na_values=['\\N'],
    )
    db.add_id_column(tu.name)
    db.create_indexes(
        tu.name,
        index_commands=[
            [('uniprot_acc', 'uniprot_mutation', 'polyphen2_hdiv_pred', 'polyphen2_hdiv_score'), True],
            [('uniprot_id', 'uniprot_mutation', 'polyphen2_hdiv_pred', 'polyphen2_hdiv_score'), True],
        ],
    )

    return t, max_id

In [ ]:
# Load the first table
chr1_t, chr1_max_id = load_table_to_db(1)

DEBUG - datapkg._helper - sed -i 's/;/,/g' '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1'
DEBUG - datapkg._helper - Running locally
DEBUG - datapkg._helper - Command ran successfully:

DEBUG - datapkg._helper - File '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant' is not compressed...
DEBUG - datapkg._helper - sed -i -e 's/\t\.\t/\t\\N\t/g' -e 's/\t\.\t/\t\\N\t/g' -e 's/^\.\t/\\N\t/g' -e 's/\t\.$/\t\\N/g' -e 's/\t\.\r$/\t\\N/g'  /kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1
DEBUG - datapkg._helper - Running locally
DEBUG - datapkg._helper - Command ran successfully:

DEBUG - datapkg._df_helper - get_file_dtypes(/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1, 100000, 100, {'sep': '\t', 'na_values': ['.', '\\N']})
DEBUG - datapkg._helper - wc -l '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.

In [ ]:
# Create the remaining tables
max_id = chr1_max_id
for suffix in list(list(range(2, 23)) + ['M', 'X', 'Y']):
    print(suffix)
    t, max_id = load_table_to_db(suffix, dtypes=chr1_t.dtypes, auto_increment=max_id+1)

DEBUG - datapkg._helper - sed -i 's/;/,/g' '/kimlab1/database_data/biodb/recipes/dbnsfp/notebooks/load_data/dbNSFP3.2a_variant.chr1'
DEBUG - datapkg._helper - Running locally


In [22]:
variant_tables = ['variant_chr{}'.format(i) for i in itertools.chain(range(1, 23), ('M', 'X', 'Y'))]
variant_tables[:2]

['variant_chr1', 'variant_chr2']

In [24]:
def create_missing_indexes(table_name):
    print(table_name); sys.stdout.flush()
    db.create_indexes(
        table_name,
        index_commands=[
            [('chr', 'pos_1based', 'ref', 'alt', 'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            [('rs_dbsnp146', 'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
            [('clinvar_rs', 'mutationassessor_uniprotid ', 'mutationassessor_variant', ), False],
        ],
    )

import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(8) as p:
    results = p.map(create_missing_indexes, variant_tables)

variant_chr1
variant_chr2
variant_chr3
variant_chr4
variant_chr5
variant_chr6
variant_chr7
variant_chr8
variant_chr9
variant_chr10
variant_chr11
variant_chr12
variant_chr13
variant_chr14
variant_chr15
variant_chr16
variant_chr17
variant_chr18
variant_chr19
variant_chr20
variant_chr21
variant_chr22
variant_chrM
variant_chrX
variant_chrY


pypy: /kimlab1/strokach/anaconda/bin/../lib/./libssl.so.1.0.0: no version information available (required by /kimlab1/strokach/anaconda/bin/../lib/libpypy-c.so)
pypy: /kimlab1/strokach/anaconda/bin/../lib/./libssl.so.1.0.0: no version information available (required by /kimlab1/strokach/anaconda/bin/../lib/libpypy-c.so)
pypy: /kimlab1/strokach/anaconda/bin/../lib/./libcrypto.so.1.0.0: no version information available (required by /kimlab1/strokach/anaconda/bin/../lib/libpypy-c.so)
^CTraceback (most recent call last):
  File "load_data/_split_variant_columns.py", line 81, in <module>
    sys.exit(main(args.file, args.idx))
  File "load_data/_split_variant_columns.py", line 60, in main
    for values in iter_row(row, uniprot_columns, variant_idx):
  File "load_data/_split_variant_columns.py", line 66, in iter_row
    for values in zip_longest(*[row[c[0] - 1].split(',') for c in columns], fillvalue='\\N'):
KeyboardInterrupt



In [71]:
# Create a MERGE table
sql_command = """\
CREATE TABLE `variant` (
  `id` bigint(20) NOT NULL AUTO_INCREMENT,
  `chr` varchar(2) DEFAULT NULL,
  `pos_1based` int(11) DEFAULT NULL,
  `ref` varchar(32) DEFAULT NULL,
  `alt` varchar(32) DEFAULT NULL,
  `aaref` varchar(32) DEFAULT NULL,
  `aaalt` varchar(32) DEFAULT NULL,
  `rs_dbsnp146` varchar(32) DEFAULT NULL,
  `hg19_chr` varchar(2) DEFAULT NULL,
  `hg19_pos_1based` int(11) DEFAULT NULL,
  `hg18_chr` varchar(2) DEFAULT NULL,
  `hg18_pos_1based` int(11) DEFAULT NULL,
  `genename` varchar(255) DEFAULT NULL,
  `cds_strand` varchar(32) DEFAULT NULL,
  `refcodon` varchar(255) DEFAULT NULL,
  `codonpos` double DEFAULT NULL,
  `codon_degeneracy` varchar(255) DEFAULT NULL,
  `ancestral_allele` varchar(32) DEFAULT NULL,
  `altai_neandertal` varchar(32) DEFAULT NULL,
  `denisova` varchar(32) DEFAULT NULL,
  `ensembl_geneid` varchar(255) DEFAULT NULL,
  `ensembl_transcriptid` mediumtext,
  `ensembl_proteinid` mediumtext,
  `aapos` varchar(255) DEFAULT NULL,
  `sift_score` varchar(255) DEFAULT NULL,
  `sift_converted_rankscore` double DEFAULT NULL,
  `sift_pred` varchar(255) DEFAULT NULL,
  `uniprot_acc_polyphen2` mediumtext,
  `uniprot_id_polyphen2` varchar(255) DEFAULT NULL,
  `uniprot_aapos_polyphen2` varchar(255) DEFAULT NULL,
  `polyphen2_hdiv_score` varchar(255) DEFAULT NULL,
  `polyphen2_hdiv_rankscore` double DEFAULT NULL,
  `polyphen2_hdiv_pred` varchar(255) DEFAULT NULL,
  `polyphen2_hvar_score` varchar(255) DEFAULT NULL,
  `polyphen2_hvar_rankscore` double DEFAULT NULL,
  `polyphen2_hvar_pred` varchar(255) DEFAULT NULL,
  `lrt_score` double DEFAULT NULL,
  `lrt_converted_rankscore` double DEFAULT NULL,
  `lrt_pred` varchar(32) DEFAULT NULL,
  `lrt_omega` double DEFAULT NULL,
  `mutationtaster_score` varchar(255) DEFAULT NULL,
  `mutationtaster_converted_rankscore` double DEFAULT NULL,
  `mutationtaster_pred` varchar(255) DEFAULT NULL,
  `mutationtaster_model` mediumtext,
  `mutationtaster_aae` mediumtext,
  `mutationassessor_uniprotid` varchar(32) DEFAULT NULL,
  `mutationassessor_variant` varchar(32) DEFAULT NULL,
  `mutationassessor_score` double DEFAULT NULL,
  `mutationassessor_score_rankscore` double DEFAULT NULL,
  `mutationassessor_pred` varchar(32) DEFAULT NULL,
  `fathmm_score` varchar(255) DEFAULT NULL,
  `fathmm_converted_rankscore` double DEFAULT NULL,
  `fathmm_pred` varchar(255) DEFAULT NULL,
  `provean_score` varchar(255) DEFAULT NULL,
  `provean_converted_rankscore` double DEFAULT NULL,
  `provean_pred` varchar(255) DEFAULT NULL,
  `transcript_id_vest3` mediumtext,
  `transcript_var_vest3` mediumtext,
  `vest3_score` mediumtext,
  `vest3_rankscore` double DEFAULT NULL,
  `metasvm_score` double DEFAULT NULL,
  `metasvm_rankscore` double DEFAULT NULL,
  `metasvm_pred` varchar(32) DEFAULT NULL,
  `metalr_score` double DEFAULT NULL,
  `metalr_rankscore` double DEFAULT NULL,
  `metalr_pred` varchar(32) DEFAULT NULL,
  `reliability_index` double DEFAULT NULL,
  `cadd_raw` double DEFAULT NULL,
  `cadd_raw_rankscore` double DEFAULT NULL,
  `cadd_phred` double DEFAULT NULL,
  `dann_score` double DEFAULT NULL,
  `dann_rankscore` double DEFAULT NULL,
  `fathmmmkl_coding_score` double DEFAULT NULL,
  `fathmmmkl_coding_rankscore` double DEFAULT NULL,
  `fathmmmkl_coding_pred` varchar(32) DEFAULT NULL,
  `fathmmmkl_coding_group` varchar(32) DEFAULT NULL,
  `eigenraw` double DEFAULT NULL,
  `eigenphred` double DEFAULT NULL,
  `eigenraw_rankscore` double DEFAULT NULL,
  `eigenpcraw` double DEFAULT NULL,
  `eigenpcraw_rankscore` double DEFAULT NULL,
  `genocanyon_score` double DEFAULT NULL,
  `genocanyon_score_rankscore` double DEFAULT NULL,
  `integrated_fitcons_score` double DEFAULT NULL,
  `integrated_fitcons_score_rankscore` double DEFAULT NULL,
  `integrated_confidence_value` double DEFAULT NULL,
  `gm12878_fitcons_score` double DEFAULT NULL,
  `gm12878_fitcons_score_rankscore` double DEFAULT NULL,
  `gm12878_confidence_value` double DEFAULT NULL,
  `h1hesc_fitcons_score` double DEFAULT NULL,
  `h1hesc_fitcons_score_rankscore` double DEFAULT NULL,
  `h1hesc_confidence_value` double DEFAULT NULL,
  `huvec_fitcons_score` double DEFAULT NULL,
  `huvec_fitcons_score_rankscore` double DEFAULT NULL,
  `huvec_confidence_value` double DEFAULT NULL,
  `gerp_nr` double DEFAULT NULL,
  `gerp_rs` double DEFAULT NULL,
  `gerp_rs_rankscore` double DEFAULT NULL,
  `phylop100way_vertebrate` double DEFAULT NULL,
  `phylop100way_vertebrate_rankscore` double DEFAULT NULL,
  `phylop20way_mammalian` double DEFAULT NULL,
  `phylop20way_mammalian_rankscore` double DEFAULT NULL,
  `phastcons100way_vertebrate` double DEFAULT NULL,
  `phastcons100way_vertebrate_rankscore` double DEFAULT NULL,
  `phastcons20way_mammalian` double DEFAULT NULL,
  `phastcons20way_mammalian_rankscore` double DEFAULT NULL,
  `siphy_29way_pi` varchar(32) DEFAULT NULL,
  `siphy_29way_logodds` double DEFAULT NULL,
  `siphy_29way_logodds_rankscore` double DEFAULT NULL,
  `gp3_ac` double DEFAULT NULL,
  `gp3_af` double DEFAULT NULL,
  `gp3_afr_ac` double DEFAULT NULL,
  `gp3_afr_af` double DEFAULT NULL,
  `gp3_eur_ac` double DEFAULT NULL,
  `gp3_eur_af` double DEFAULT NULL,
  `gp3_amr_ac` double DEFAULT NULL,
  `gp3_amr_af` double DEFAULT NULL,
  `gp3_eas_ac` double DEFAULT NULL,
  `gp3_eas_af` double DEFAULT NULL,
  `gp3_sas_ac` double DEFAULT NULL,
  `gp3_sas_af` double DEFAULT NULL,
  `twinsuk_ac` double DEFAULT NULL,
  `twinsuk_af` double DEFAULT NULL,
  `alspac_ac` double DEFAULT NULL,
  `alspac_af` double DEFAULT NULL,
  `esp6500_aa_ac` double DEFAULT NULL,
  `esp6500_aa_af` double DEFAULT NULL,
  `esp6500_ea_ac` double DEFAULT NULL,
  `esp6500_ea_af` double DEFAULT NULL,
  `exac_ac` double DEFAULT NULL,
  `exac_af` double DEFAULT NULL,
  `exac_adj_ac` double DEFAULT NULL,
  `exac_adj_af` double DEFAULT NULL,
  `exac_afr_ac` double DEFAULT NULL,
  `exac_afr_af` double DEFAULT NULL,
  `exac_amr_ac` double DEFAULT NULL,
  `exac_amr_af` double DEFAULT NULL,
  `exac_eas_ac` double DEFAULT NULL,
  `exac_eas_af` double DEFAULT NULL,
  `exac_fin_ac` double DEFAULT NULL,
  `exac_fin_af` double DEFAULT NULL,
  `exac_nfe_ac` double DEFAULT NULL,
  `exac_nfe_af` double DEFAULT NULL,
  `exac_sas_ac` double DEFAULT NULL,
  `exac_sas_af` double DEFAULT NULL,
  `exac_nontcga_ac` double DEFAULT NULL,
  `exac_nontcga_af` double DEFAULT NULL,
  `exac_nontcga_adj_ac` double DEFAULT NULL,
  `exac_nontcga_adj_af` double DEFAULT NULL,
  `exac_nontcga_afr_ac` double DEFAULT NULL,
  `exac_nontcga_afr_af` double DEFAULT NULL,
  `exac_nontcga_amr_ac` double DEFAULT NULL,
  `exac_nontcga_amr_af` double DEFAULT NULL,
  `exac_nontcga_eas_ac` double DEFAULT NULL,
  `exac_nontcga_eas_af` double DEFAULT NULL,
  `exac_nontcga_fin_ac` double DEFAULT NULL,
  `exac_nontcga_fin_af` double DEFAULT NULL,
  `exac_nontcga_nfe_ac` double DEFAULT NULL,
  `exac_nontcga_nfe_af` double DEFAULT NULL,
  `exac_nontcga_sas_ac` double DEFAULT NULL,
  `exac_nontcga_sas_af` double DEFAULT NULL,
  `exac_nonpsych_ac` double DEFAULT NULL,
  `exac_nonpsych_af` double DEFAULT NULL,
  `exac_nonpsych_adj_ac` double DEFAULT NULL,
  `exac_nonpsych_adj_af` double DEFAULT NULL,
  `exac_nonpsych_afr_ac` double DEFAULT NULL,
  `exac_nonpsych_afr_af` double DEFAULT NULL,
  `exac_nonpsych_amr_ac` double DEFAULT NULL,
  `exac_nonpsych_amr_af` double DEFAULT NULL,
  `exac_nonpsych_eas_ac` double DEFAULT NULL,
  `exac_nonpsych_eas_af` double DEFAULT NULL,
  `exac_nonpsych_fin_ac` double DEFAULT NULL,
  `exac_nonpsych_fin_af` double DEFAULT NULL,
  `exac_nonpsych_nfe_ac` double DEFAULT NULL,
  `exac_nonpsych_nfe_af` double DEFAULT NULL,
  `exac_nonpsych_sas_ac` double DEFAULT NULL,
  `exac_nonpsych_sas_af` double DEFAULT NULL,
  `clinvar_rs` varchar(32) DEFAULT NULL,
  `clinvar_clnsig` varchar(32) DEFAULT NULL,
  `clinvar_trait` mediumtext,
  `clinvar_golden_stars` varchar(32) DEFAULT NULL,
  `interpro_domain` mediumtext,
  `gtex_v6_gene` mediumtext,
  `gtex_v6_tissue` mediumtext,
  PRIMARY KEY (`id`),
  KEY `a` (`chr`,`pos_1based`,`ref`,`alt`),
  KEY `b` (`chr`,`pos_1based`,`aaref`,`aaalt`),
  KEY `c` (`rs_dbsnp146`),
  KEY `d` (`aaref`,`aaalt`),
  KEY `e` (`ensembl_geneid`,`aaref`,`aaalt`),
  KEY `f` (`genename`),
  KEY `g` (`uniprot_id_polyphen2`,`uniprot_aapos_polyphen2`),
  KEY `h` (`mutationassessor_uniprotid`,`mutationassessor_variant`),
  KEY `i` (`hg19_chr`,`hg19_pos_1based`,`ref`,`alt`),
  KEY `j` (`hg18_chr`,`hg18_pos_1based`,`ref`,`alt`),
  KEY `k` (`chr`,`pos_1based`,`ref`,`alt`,`mutationassessor_uniprotid`,`mutationassessor_variant`),
  KEY `l` (`rs_dbsnp146`,`mutationassessor_uniprotid`,`mutationassessor_variant`),
  KEY `m` (`clinvar_rs`,`mutationassessor_uniprotid`,`mutationassessor_variant`)
) ENGINE=MERGE 
  UNION=(variant_chr1, variant_chr2, variant_chr3, variant_chr4, variant_chr5, variant_chr6, variant_chr7, 
         variant_chr8, variant_chr9, variant_chr10, variant_chr11, variant_chr12, variant_chr13, variant_chr14, 
         variant_chr15, variant_chr16, variant_chr17, variant_chr18, variant_chr19, variant_chr20, variant_chr21, 
         variant_chr22, variant_chrM, variant_chrX, variant_chrY)
  DEFAULT CHARSET=latin1;
"""
print(sql_command)

CREATE TABLE `variant` (
  `id` bigint(20) NOT NULL AUTO_INCREMENT,
  `chr` varchar(2) DEFAULT NULL,
  `pos_1based` int(11) DEFAULT NULL,
  `ref` varchar(32) DEFAULT NULL,
  `alt` varchar(32) DEFAULT NULL,
  `aaref` varchar(32) DEFAULT NULL,
  `aaalt` varchar(32) DEFAULT NULL,
  `rs_dbsnp146` varchar(32) DEFAULT NULL,
  `hg19_chr` varchar(2) DEFAULT NULL,
  `hg19_pos_1based` int(11) DEFAULT NULL,
  `hg18_chr` varchar(2) DEFAULT NULL,
  `hg18_pos_1based` int(11) DEFAULT NULL,
  `genename` varchar(255) DEFAULT NULL,
  `cds_strand` varchar(32) DEFAULT NULL,
  `refcodon` varchar(255) DEFAULT NULL,
  `codonpos` double DEFAULT NULL,
  `codon_degeneracy` varchar(255) DEFAULT NULL,
  `ancestral_allele` varchar(32) DEFAULT NULL,
  `altai_neandertal` varchar(32) DEFAULT NULL,
  `denisova` varchar(32) DEFAULT NULL,
  `ensembl_geneid` varchar(255) DEFAULT NULL,
  `ensembl_transcriptid` mediumtext,
  `ensembl_proteinid` mediumtext,
  `aapos` varchar(255) DEFAULT NULL,
  `sift_score` varchar(255) DE

In [72]:
db.engine.execute(sql_command)

# Compress

In [38]:
mysqld.myisampack(SCHEMA_NAME)

AttributeError: 'MySQLDaemon' object has no attribute 'data_dir'